In [1]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

In [2]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [3]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

In [4]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

(Wrapper)>[PointerTensor | me:17656813072 -> bob:11974274660]

In [5]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

(Wrapper)>[PointerTensor | me:53077959453 -> alice:17656813072]

In [6]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{11974274660: tensor([1, 2, 3, 4])}

In [7]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

{17656813072: (Wrapper)>[PointerTensor | alice:17656813072 -> bob:11974274660]}

In [8]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

(Wrapper)>[PointerTensor | me:17656813072 -> bob:11974274660]

In [9]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

tensor([1, 2, 3, 4])

In [10]:
bob._objects

{}

In [11]:
alice._objects

{}

In [12]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

In [13]:
y = p2p2x + p2p2x

In [14]:
y

(Wrapper)>[PointerTensor | me:67708914879 -> alice:28710807416]

In [15]:
bob._objects

{66640927888: tensor([1, 2, 3, 4, 5]),
 47792284664: tensor([ 2,  4,  6,  8, 10])}

In [16]:
alice._objects

{65358832729: (Wrapper)>[PointerTensor | alice:65358832729 -> bob:66640927888],
 28710807416: (Wrapper)>[PointerTensor | alice:28710807416 -> bob:47792284664]}

In [17]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [18]:
bob._objects

{66640927888: tensor([1, 2, 3, 4, 5])}

In [19]:
alice._objects

{65358832729: (Wrapper)>[PointerTensor | alice:65358832729 -> bob:66640927888]}

In [20]:
p2p2x.get().get()

tensor([1, 2, 3, 4, 5])

In [21]:
bob._objects

{}

In [22]:
alice._objects

{}

In [23]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [24]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {44536264618: tensor([1, 2, 3, 4, 5])}
alice: {}


In [25]:
x = x.move(alice)   # x是bob的数据，通过move()操作可以直接由bob将x发送给alice（注意，send()和get()是本地机器才有的操作）

In [26]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {}
alice: {44536264618: tensor([1, 2, 3, 4, 5])}


In [27]:
x

(Wrapper)>[PointerTensor | me:11313716795 -> alice:44536264618]